In [17]:
import os
import json
import copy
import requests
import pandas as pd
import time

In [130]:
df = pd.read_csv('../Data/leetcode_problems.csv', index_col='question_id')
slugs = list(df['question__title_slug'])
s = requests.Session()
template = {"operationName":"questionData","variables":{"titleSlug":None},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    __typename\n  }\n}\n"}

In [34]:
def slug_to_qid(slug: str):
    search = df[df['question__title_slug'] == slug]
    return None if len(search) == 0 else int(search['question_id'])

In [120]:
def run(slug):
    payload = {"operationName":"questionData","variables":{"titleSlug":slug},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    __typename\n  }\n}\n"}
    payload_str = json.dumps(payload)
    headers = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36",
        "content-length": str(len(payload_str)),
        "content-type": "application/json",
        # "accept-encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Accept": "*/*",
        "origin": "https://leetcode.com",
        "referer": "https://leetcode.com/problems/two-sum/"
    }
    response = s.post("https://leetcode.com/graphql", headers=headers, json=payload)
    question = response.json()['data']['question']
    question_id = int(question['questionId'])
    similar_question_ids = [slug_to_qid(problem['titleSlug']) for problem in json.loads(question['similarQuestions'])]
    topicTags = [{'name': tag['name'], 'slug': tag['slug']} for tag in question['topicTags']]
    return {
        "question_id": question_id, 
        "topicTags": topicTags, 
        "similar_question_ids": similar_question_ids,
        "likes": question["likes"],
        "dislikes": question["dislikes"]
    }

In [121]:
def main():
    result = []
    for index, slug in enumerate(slugs):
        if index > 5:
            break
        result.append(run(slug))
    return result

In [122]:
results = main()

In [123]:
list(results)

[{'question_id': 1,
  'topicTags': [{'name': 'Array', 'slug': 'array'},
   {'name': 'Hash Table', 'slug': 'hash-table'}],
  'similar_question_ids': [15, 18, 167, 170, 560, 653, 1083],
  'likes': 16782,
  'dislikes': 609},
 {'question_id': 2,
  'topicTags': [{'name': 'Linked List', 'slug': 'linked-list'},
   {'name': 'Math', 'slug': 'math'}],
  'similar_question_ids': [43, 67, 371, 415, 445, 1031],
  'likes': 9183,
  'dislikes': 2309},
 {'question_id': 3,
  'topicTags': [{'name': 'Hash Table', 'slug': 'hash-table'},
   {'name': 'Two Pointers', 'slug': 'two-pointers'},
   {'name': 'String', 'slug': 'string'},
   {'name': 'Sliding Window', 'slug': 'sliding-window'}],
  'similar_question_ids': [159, 340, 1034],
  'likes': 10653,
  'dislikes': 607},
 {'question_id': 4,
  'topicTags': [{'name': 'Array', 'slug': 'array'},
   {'name': 'Binary Search', 'slug': 'binary-search'},
   {'name': 'Divide and Conquer', 'slug': 'divide-and-conquer'}],
  'similar_question_ids': [],
  'likes': 7834,
  'di

In [133]:
for question in results:
    df.loc[question['question_id'], 'likes'] = question['likes']
    df.loc[question['question_id'], 'dislikes'] = question['dislikes']
    df.loc[question['question_id'], 'topicTags'] = str(question['topicTags'])
    df.loc[question['question_id'], 'similar_question_ids'] = str(question['similar_question_ids'])

In [134]:
print(df.loc[1, 'likes'])

16782.0


In [136]:
df.to_csv("../Data/leetcode_problems.csv")